## Javascript advanced 

In [ ]:
'use strict';

### Prototype inheritance

JavaScript objects are dynamic "bags" of properties (referred to as own properties).

In [ ]:
const gigel = {
    a: 1,
    b: 2
};
console.log(Object.getOwnPropertyNames(gigel));

gigel.c = 3;
console.log(Object.getOwnPropertyNames(gigel));

delete gigel.b;
console.log(Object.getOwnPropertyNames(gigel));

JavaScript objects have a link to a prototype object. 
When trying to access a property of an object, the property will not only be sought on the object but on the prototype of the object and recursively on its ancestors.

In [ ]:
function OldSchoolJsClass() {
    this.a = 1;
    this.b = 2;
}
/**
 * BIG WARNING: This is the func.prototype property of functions != Object.getPrototypeOf() of objects.
 * It specifies the [[Prototype]] to be assigned to all instances of objects 
 * created by the given function when used as a constructor.
 */
console.log('OldSchoolJsClass.prototype != Object.getPrototypeOf(OldSchoolJsClass)');
console.log(OldSchoolJsClass.prototype);
console.log(Object.getPrototypeOf(OldSchoolJsClass).toString());
console.log(OldSchoolJsClass.__proto__.toString());

OldSchoolJsClass.prototype.b = 3;
OldSchoolJsClass.prototype.c = 4;

In [ ]:
const ionel = new OldSchoolJsClass();

console.log(Object.getOwnPropertyNames(ionel));
console.log(Object.getPrototypeOf(ionel));

console.log(`a=${ionel.a}, b=${ionel.b}, c=${ionel.c}`);

JavaScript does not have "methods" in the form that class-based languages define them. In JavaScript, any function can be added to an object in the form of a property.

In [ ]:
OldSchoolJsClass.prototype.incrementA = function (value) {
    console.log(`[BEFORE]: this.a=${this.a}`);
    this.a += value;
    console.log(`[AFTER]: this.a=${this.a}`);
};

ionel.incrementA(5);

// Extremly frequent mistake
setTimeout(/* function: */ ionel.incrementA, /* delay: */ 1000, /* arg1: */ 5);

// Always bind the method to the instance before passing it as a reference
setTimeout(ionel.incrementA.bind(ionel), 1000, 5);

In [ ]:
function AnotherOldJsClass() {
    this.x = 1;
    this.y = 2;
    this.a = 10;
}

const costel = new AnotherOldJsClass();
OldSchoolJsClass.prototype.incrementA.call(costel, 5);
// why calling costel.incrementA(5) doesn't work?

console.log(`costel.a=${costel.a}`);

In [ ]:
function InheritedOldJsClass() {
    this.z = 1;
    this.w = 2;
}

const inheritedPropertiesAndMethods = Object.create(OldSchoolJsClass.prototype);
console.log(Object.getOwnPropertyNames(inheritedPropertiesAndMethods));
console.log(Object.getOwnPropertyNames(inheritedPropertiesAndMethods.__proto__));

InheritedOldJsClass.prototype = inheritedPropertiesAndMethods;
InheritedOldJsClass.prototype.c = 5;

const cornel = new InheritedOldJsClass();

console.log(Object.getOwnPropertyNames(cornel));
// aka Object.getOwnPropertyNames(cornel.__proto__)
console.log(Object.getOwnPropertyNames(Object.getPrototypeOf(cornel)));
console.log(Object.getOwnPropertyNames(cornel.__proto__.__proto__));

console.log(`z=${cornel.z}, w=${cornel.w}, b=${cornel.b}, c=${cornel.c}`);

cornel.incrementA(5); // why is it printing this?

In [ ]:
class ParentClass {

    constructor() {
        this.a = 1;
        this.b = 2;
        this.c = 4;
    }

    incrementA(value) {
        console.log(`[BEFORE]: this.a=${this.a}`);
        this.a += value;
        console.log(`[AFTER]: this.a=${this.a}`);
    }
}

class ChildClass extends ParentClass {
    
    constructor() {
        super();

        this.x = 1;
        this.y = 2;
        this.a = 10;
    }
}

const parentInstance = new ParentClass();
console.log(Object.getOwnPropertyNames(parentInstance));
// aka Object.getOwnPropertyNames(parentInstance.__proto___)
console.log(Object.getOwnPropertyNames(Object.getPrototypeOf(parentInstance))); 

const childInstance = new ChildClass();
console.log(Object.getOwnPropertyNames(childInstance));
// aka Object.getOwnPropertyNames(childInstance.__proto___)
console.log(Object.getOwnPropertyNames(Object.getPrototypeOf(childInstance)));

In [ ]:
parentInstance.incrementA(5);
console.log(`parent.a = ${parentInstance.a}`);

childInstance.incrementA(5);
console.log(`child.a = ${childInstance.a}`);

// Always bind the method to the instance before passing it as a reference
// Also, spot the binding
setTimeout(parentInstance.incrementA.bind(childInstance), 1000, 5);

### Arrow functions

From now on I'll mainly use arrow functions.

The main take away:
* Should not be used as methods
* Not suitable for call, apply and bind functions

They are much nicer to use, but take care that they capture 'this' at the moment they are defined, as opposed to regular functions that can be binded at call time to whatever 'this' you like.

For more details read https://developer.mozilla.org/en-US/docs/Web/JavaScript/Reference/Functions/Arrow_functions


### Callbaks vs Promises vs async / await

A Promise is a proxy for a value not necessarily known when the promise is created.

It's the modern replacement for the callback pattern, known also as the callback hell. 

I'm wondering why?

In [ ]:
function doSomething(someArg, callback) {
    setTimeout(/* function: */callback, /* delay: */1000, /* arg1 */null, /* arg2: */`${someArg}: great success`);
}

function doSomethingElse(someArg, callback) {
    setTimeout(/* function: */callback, /* delay: */1000, /* arg1 */null, /* arg2: */`${someArg}: not good, not terrible`);
}

function dropItLikeItsHot(callback) {
    setTimeout(/* function: */callback, /* delay: */1000, /* arg1 */new Error('wrong pocket, my son'), /* arg2: */null);
}

function hardToImplementCorrectly(someArg, callback) {
    
    let endResult = '';
    let errorEncountered = null;
    let asyncCallsToComplete = 3;
    
    doSomething(someArg, (err, result) => {
    
        --asyncCallsToComplete;
        
        if (err) {
            callback(err, null);
            return; // early return because operations below depend on this result
        }
        
        endResult += result;

        doSomethingElse(result, (err, result) => {

            if (err) {
                errorEncountered = err;
            } else {
                endResult += result;
            }
            
            if (--asyncCallsToComplete === 0) {
                
                if (err) {
                    callback(errorEncountered, null);
                } else {
                    callback(null, endResult);
                }
            }
        });
        
        dropItLikeItsHot((err, result) => {

            if (err) {
                errorEncountered = err;
            } else {
                endResult += result;
            }
            
            if (--asyncCallsToComplete === 0) {
                
                if (err) {
                    callback(errorEncountered, null);
                } else {
                    callback(null, endResult);
                }
            }
        });    
    });
}

hardToImplementCorrectly('Borat', (err, res) => {
    if (err) {
        console.error(err);
        return;
    }
    console.info(res);
});

Promises simplify control flow, although not enough, by chaining callbacks instead of nesting them.

In [ ]:
function doSomething(someArg) {

    return new Promise((resolve, reject) => {
        setTimeout(/* function: */resolve, /* delay: */1000, /* arg: */`${someArg}: great success`);
    });
}

function doSomethingElse(someArg) {
    
    return new Promise((resolve, reject) => {
        setTimeout(/* function: */resolve, /* delay: */1000, /* arg: */`${someArg}: not good, not terrible`);
    });
}

function dropItLikeItsHot() {
    
    return new Promise((resolve, reject) => {
         setTimeout(/* function: */reject, /* delay: */1000, /* arg */new Error('wrong pocket, my son'));
    });
}

function easierToImplementCorrectly(someArg) {
    
    return doSomething(someArg)
        .then((result) => {

            return Promise.all([
                doSomethingElse(result),
                dropItLikeItsHot()
            ]);
        })
        .then(([resultSomethingElse, resultDropItLikeItsHot]) => resultSomethingElse + resultDropItLikeItsHot);
}

easierToImplementCorrectly('Borat')
    .then(result => console.log(result))
    .catch(err => console.error(err));

The most recent simplification to Promises API is async / await, which allows to suspend an async function execution while waiting for a Promise to either resolve or reject.

In this way, you can write code in an imperative manner, like you would do when using synchronous APIs.

Also, it allows to perform error handling with try { } catch { } blocks that, again, resemble to synchronous implementations. 

In [ ]:
async function stronglyRecommendedImplementation(someArg) {
    
    const result = await doSomething(someArg);
    const [resultSomethingElse, resultDropItLikeItsHot] = await Promise.all([
        doSomethingElse(result),
        dropItLikeItsHot()
    ]);
    return resultSomethingElse + resultDropItLikeItsHot;
}

(async () => { // this is because we cannot use await keyword top level, just in async functions
    
    try {
        await stronglyRecommendedImplementation('Borat');
    } catch (err) {
        console.error(err);
    }
    
})();